In [22]:
%%capture
! wget https://raw.githubusercontent.com/zadushevno/nlp-4-project/main/aspect_extraction/test_aspects.txt
! wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_reviews.txt
! wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_aspects.txt

In [29]:
import pandas as pd
import csv

In [10]:
texts = pd.read_csv('dev_reviews.txt', delimiter='\t', names = ['review_id', 'text'])
texts.head(5)

,review_id,text
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ..."
1,1427,Здравствуйте!Посетили ваше заведение вчера пер...
2,16714,"Были в пятницу (19.03.10), заказывали столик д..."
3,797,"Были в ресторане 2 раза. Один раз днем, все по..."
4,34710,Удивляюсь отзывам про хорошее обслуживание. Бы...


In [12]:
test_aspects = pd.read_csv('aspects.txt', delimiter='\t', names = ['review_id', 'category', 'mention', 'start', 'stop'])
test_aspects.head(5)

,review_id,category,mention,start,stop
0,13823,Whole,аппетит,8,15
1,13823,Service,встретил,138,146
2,13823,Service,менеджер,147,155
3,13823,Service,девушка,179,186
4,13823,Service,проводила к столу,188,205


In [14]:
data = pd.merge(texts, test_aspects, on='review_id')
data.head(5)

,review_id,text,category,mention,start,stop
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Whole,аппетит,8,15
1,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,встретил,138,146
2,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,менеджер,147,155
3,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,девушка,179,186
4,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,проводила к столу,188,205


In [15]:
data['totrain'] = '@ ' + data['mention'] + ' @ ' + data['text']
data.head(5)

,review_id,text,category,mention,start,stop,totrain
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Whole,аппетит,8,15,"@ аппетит @ Зашли в""аппетит"" случайно.Не смотр..."
1,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,встретил,138,146,"@ встретил @ Зашли в""аппетит"" случайно.Не смот..."
2,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,менеджер,147,155,"@ менеджер @ Зашли в""аппетит"" случайно.Не смот..."
3,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,девушка,179,186,"@ девушка @ Зашли в""аппетит"" случайно.Не смотр..."
4,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,проводила к столу,188,205,"@ проводила к столу @ Зашли в""аппетит"" случайн..."


In [18]:
sentences = data.totrain.values

Чтобы скачать преобученную модель из [нашей папки](https://drive.google.com/drive/u/0/folders/1e8WjOU0U33uCS4XWpkWmb2F6G2aMXak2), нужно добавить ярлык папки к себе в гугл драйв и запустить код

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/ruBert-base')
pred_model = AutoModelForSequenceClassification.from_pretrained('/content/gdrive/MyDrive/absa-project').to('cuda')

In [4]:
from transformers import pipeline
from transformers import AutoConfig

In [5]:
pipe = pipeline('text-classification', model=pred_model, tokenizer=tokenizer, device='cuda')

In [19]:
mapping = {
    "LABEL_0":"neutral",
    "LABEL_1":"positive",
    "LABEL_2":"both",
    "LABEL_3":"negative"
}

In [ ]:
labels = []
for sentence in sentences:
    labels.append(mapping[pipe(sentence)[0]['label']])

In [24]:
data['labels'] = labels
data.head(5)

,review_id,text,category,mention,start,stop,totrain,labels
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Whole,аппетит,8,15,"@ аппетит @ Зашли в""аппетит"" случайно.Не смотр...",neutral
1,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,встретил,138,146,"@ встретил @ Зашли в""аппетит"" случайно.Не смот...",neutral
2,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,менеджер,147,155,"@ менеджер @ Зашли в""аппетит"" случайно.Не смот...",neutral
3,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,девушка,179,186,"@ девушка @ Зашли в""аппетит"" случайно.Не смотр...",neutral
4,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,проводила к столу,188,205,"@ проводила к столу @ Зашли в""аппетит"" случайн...",neutral


In [36]:
data = data.drop(['text', 'totrain'], axis=1)
data.head(5)

,review_id,category,mention,start,stop,labels
0,13823,Whole,аппетит,8,15,neutral
1,13823,Service,встретил,138,146,neutral
2,13823,Service,менеджер,147,155,neutral
3,13823,Service,девушка,179,186,neutral
4,13823,Service,проводила к столу,188,205,neutral


In [37]:
data.to_csv('test_aspects_with_sent.txt', sep='\t', index=False)